2.5. Uruchom klienta BigQuery z poziomu Pythona.

In [1]:
import os
import pandas as pd
from google.cloud import bigquery
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "C:\\Users\\rober\\OneDrive\\Pulpit\\semestr 6\\BigData\\windy-access-416710-5959edbfa345.json"  # lokalizacja pobranego klucza z punktu 1.4.
client = bigquery.Client()

DefaultCredentialsError: File C:\Users\rober\OneDrive\Pulpit\semestr 6\BigData\windy-access-416710-5959edbfa345.json was not found.

 2.6. Pobierz dane do obiektu DataFrame.

In [ ]:
query = "SELECT * FROM `bigquery-public-data.covid19_open_data.covid19_open_data` LIMIT 10"
query_job = client.query(query)
query_result = query_job.result()
df = query_result.to_dataframe()
df.to_csv('C:\\Users\\rober\\OneDrive\\Pulpit\\semestr 6\\BigData\\zad1\\wyniki\\dane.csv', index=False)

Część 4

Przeanalizuj poniższe przypadki. Zastanów się, jakie dane potrzebujesz do każdego z nich, a następnie zapisz je w osobnych, jak najprostszych obiektach DataFrame. Na tym etapie nie przetwarzaj ich. Zadbaj o czystość danych, m.in. usuń niepotrzebne puste wartości i duplikaty, zunifikuj sposób podawania informacji, zaproponuj sposób naprawy błędnych danych. Gotowe dane z obiektów DataFrame zapisz w osobnych plikach CSV.

4.1. Chcemy posiadać podstawowe dane o wszystkich krajach świata, które będą zrozumiałe dla człowieka oraz uniwersalne i potencjalnie przyszłościowe do dalszego przetwarzania.

In [ ]:
query = "SELECT DISTINCT country_name AS `Country name`, iso_3166_1_alpha_3 AS `3 letter country code` FROM `bigquery-public-data.covid19_open_data.covid19_open_data`"
query_country = client.query(query)
query_country_result = query_country.result()
country_df = query_country_result.to_dataframe()
country_df.dropna(inplace=True)
country_df.to_csv('C:\\Users\\rober\\OneDrive\\Pulpit\\semestr 6\\BigData\\zad1\\wyniki\\country_data.csv', index=False)

4.2. Chcemy wygenerować statystyki dotyczące zachorowań na COVID-19 na całym świecie.

In [ ]:
query = "SELECT country_name AS `Country name`, iso_3166_1_alpha_3 AS `3 letter country code`, SUM(new_confirmed) as `Total confirmed cases` FROM `bigquery-public-data.covid19_open_data.covid19_open_data`  GROUP BY country_name, iso_3166_1_alpha_3 ORDER BY country_name "
query_disease = client.query(query)
query_disease_result = query_disease.result()
disease_df = query_disease_result.to_dataframe()
disease_df.dropna(inplace=True)
disease_df.to_csv('C:\\Users\\rober\\OneDrive\\Pulpit\\semestr 6\\BigData\\zad1\\wyniki\\disease_data.csv', index=False)

4.3. Chcemy poznać efekty COVID-19 poprzez uwypuklenie problemu śmiertelności ludzi spowodowanej wirusem.

In [ ]:
query = "SELECT country_name AS `Country name`, iso_3166_1_alpha_3 AS `3 letter country code`, SUM(new_deceased) as `Total deceased` FROM `bigquery-public-data.covid19_open_data.covid19_open_data`  GROUP BY country_name, iso_3166_1_alpha_3 ORDER BY country_name "
query_deceased = client.query(query)
query_deceased_result = query_deceased.result()
deceased_df = query_deceased_result.to_dataframe()
deceased_df.dropna(inplace=True)
deceased_df.to_csv('C:\\Users\\rober\\OneDrive\\Pulpit\\semestr 6\\BigData\\zad1\\wyniki\\deceased_data.csv', index=False)

4.4. Chcemy zaobserwować trendy i zależności dotyczące szczepień na COVID-19.

In [ ]:
query = "SELECT country_name AS `Country name`, iso_3166_1_alpha_3 AS `3 letter country code`, SUM(new_persons_fully_vaccinated) as `Total persons fully vaccinated`, SUM(new_persons_vaccinated) as `Total vacinations` FROM `bigquery-public-data.covid19_open_data.covid19_open_data`  GROUP BY country_name, iso_3166_1_alpha_3 ORDER BY country_name "
query_vaccine = client.query(query)
query_vaccine_result = query_vaccine.result()
vaccine_df = query_vaccine_result.to_dataframe()
vaccine_df.dropna(inplace=True)
vaccine_df.to_csv('C:\\Users\\rober\\OneDrive\\Pulpit\\semestr 6\\BigData\\zad1\\wyniki\\vaccine_data.csv', index=False)

4.5. Zdefiniuj własny dodatkowy przypadek.

Liczba wykonanych testów na całym świecie

In [ ]:
query = "SELECT country_name AS `Country name`, iso_3166_1_alpha_3 AS `3 letter country code`, SUM(cast(new_tested as integer)) as `Total tested persons` FROM `bigquery-public-data.covid19_open_data.covid19_open_data`  GROUP BY country_name, iso_3166_1_alpha_3 ORDER BY country_name "
query_tested = client.query(query)
query_tested_result = query_tested.result()
tested_df = query_tested_result.to_dataframe()
tested_df.dropna(inplace=True)
tested_df.to_csv('C:\\Users\\rober\\OneDrive\\Pulpit\\semestr 6\\BigData\\zad1\\wyniki\\tested_data.csv', index=False)

Część 5

Połącz ze sobą wszystkie dane otrzymane w części 4. Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [ ]:
merged_country_disease = pd.merge(country_df, disease_df, on=['Country name', '3 letter country code'])
merged_country_disease_deceased = pd.merge(merged_country_disease, deceased_df, on=['Country name', '3 letter country code'])
merged_country_disease_deceased_vaccine = pd.merge(merged_country_disease_deceased, vaccine_df, on=['Country name', '3 letter country code'])
merged_df_per_country = pd.merge(merged_country_disease_deceased_vaccine, tested_df, on=['Country name', '3 letter country code'])
merged_df_per_country.to_csv('C:\\Users\\rober\\OneDrive\\Pulpit\\semestr 6\\BigData\\zad1\\wyniki\\part_5_merged.csv', index=False)

Część 6

Połącz ze sobą dane otrzymane w części 5 oraz dane znajdujące się w plikach:
- world_countries.csv
- gdp.csv
Nowy zbiór danych zapisz jako oddzielny obiekt DataFrame. Jeżeli uważasz, że należy stworzyć kilka takich obiektów, zrób to i zapisz swoje uzasadnienie. Pamiętaj o ciągłej konieczności zachowania czystości danych. Wynik tego zadania zapisz w pliku / plikach CSV.

In [2]:
world_countries_df = pd.read_csv('C:\\Users\\rober\\OneDrive\\Pulpit\\semestr 6\\BigData\\zad1\\dane_z_wikamp\\world_countries.csv')
gdp_df = pd.read_csv('C:\\Users\\rober\\OneDrive\\Pulpit\\semestr 6\\BigData\\zad1\\dane_z_wikamp\\gdp.csv')

merged_df_per_country_world_countries = pd.merge(merged_df_per_country, world_countries_df, left_on='Country name', right_on='Country/Territory')
merged_df_per_country_world_countries.drop('Country/Territory', axis=1, inplace=True)
merged_df_per_country_world_countries.drop('CCA3', axis=1, inplace=True)
merged_df_per_country_world_countries.to_csv('C:\\Users\\rober\\OneDrive\\Pulpit\\semestr 6\\BigData\\zad1\\wyniki\\part_6_05_merged.csv', index=False)

merged_df_data = pd.merge(merged_df_per_country_world_countries, gdp_df, left_on='Country name', right_on='Country Name')
merged_df_data.drop('Country Name', axis=1, inplace=True)
merged_df_data.drop('Country Code', axis=1, inplace=True)
merged_df_data.rename(columns={"Value" : "GDP Value"}, inplace=True)
merged_df_data.to_csv('C:\\Users\\Kacper\\Documents\\Studia\\6sem\\BigDataWyniki\\Zad1\\part_6_01_merged.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\rober\\OneDrive\\Pulpit\\semestr 6\\BigData\\zad1\\dane_z_wikamp\\world_countries.csv'